# Importando as biblitecas

In [ ]:
import pandas as pd
import re
import numpy as np

# Lendo o arquivo de mortalidade

In [ ]:
Mortalidade_2019 = pd.read_csv(r'C:\Projeto Final\data\Mortalidade_Geral_2019.csv', sep = ';')

# Estudando a tabela 

In [ ]:
Mortalidade_2019.head()

In [ ]:
Mortalidade_2019.info()

In [ ]:
Mortalidade_2019.columns

# Fazendo Verificação e Limpeza da Tabela

In [ ]:
Mortalidade_2019 = Mortalidade_2019.loc[:,['DTOBITO', 'CODMUNOCOR', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'CAUSABAS']]

Vendo os gêneros que mais sofreram com as mortes

In [ ]:
Mortalidade_2019['SEXO'].value_counts()

In [ ]:
#Ao verificar que o Sexo que está igual a 1 possui maior valor, pode-se confirmar que são os homens, pois ao pesquisar sobre a mortalidade de gêneros, os homens são os que mais sofrem mortes

Corrigindo a coluna de idade

In [ ]:
Mortalidade_2019['IDADE'] = Mortalidade_2019['IDADE'].astype(str)

def limpeza_idade(linha):
    x = linha['IDADE']
    if linha['DTOBITO']  == linha['DTNASC'] or linha['DTOBITO'] == linha['DTNASC']+1:
        return 0

    elif x.startswith('5') and len(x) == 3:
        x_new_5 = x.replace('5', '1', 1)
        x_new_5 = float(x)
        return x_new_5

    elif x.startswith('4') and len(x) == 3:
        x_new_4 = float(x[1:])
        return x_new_4

    elif x.startswith('3') and len(x) == 3 or len(x) == 2:
        x_new_3 = float(x[1:])
        x_new_3 = (x_new_3 * 833334/100)
        return x_new_3

    elif x.startswith('2') and len(x) == 3 or len(x) == 2:
        x_new_2 = float(x[1:])
        x_new_2 = (x_new_2 * 114155/10000)
        return x_new_2

    elif x.startswith('1') and len(x) == 3 or len(x) == 2:
        x_new_1 = float(x[1:])
        x_new_1 = (x_new_1 * 1.90259/1000000)
        return x_new_1

Mortalidade_2019['IDADE'] = Mortalidade_2019.apply(limpeza_idade, axis = 1)

Mudando o formato das datas da coluna de óbito e nascimento

In [ ]:
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].astype(str).str.zfill(8)                          # deixa todas as linhas com 8 números, alem de transformar em str
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].apply(lambda x: f'{x[0:2]}/{x[2:4]}/{x[4:]}')     # aplica '/' em todas as datas
Mortalidade_2019['DTOBITO'] = pd.to_datetime(Mortalidade_2019['DTOBITO'])                                   # transforma em datetime

In [ ]:
Mortalidade_2019['DTNASC'] = Mortalidade_2019['DTNASC'].fillna(0)                      #transforma nulo em zeros
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC']).dt.date        #transforma em data
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC'])

Arrumando a coluna de Causas para conectar a outra tabela

In [ ]:
Mortalidade_2019['CAUSABAS'] = Mortalidade_2019['CAUSABAS'].apply(lambda x: f'{x[0:3]}')

# Lendo arquivo CID para juntar as tabelas de CID e Mortalidade

In [ ]:
CID_2019 = pd.read_excel(r'C:\Projeto Final\data\CID_2019.xlsx', header=15, skipfooter=8)

# Fazendo o tratamento do arquivo CID

In [ ]:
CID_2019['Total'] = CID_2019['Total'].apply(lambda x: f'{x[0:3]}')
CID_2019 = CID_2019.loc[:,['Total']]
CID_2019  = CID_2019.drop(CID_2019.loc[CID_2019['Total']=='Cap'].index)
CID_2019['Classificação de causas'] = CID_2019['Total']

Classificação de causas (coluna CAUSABAS)

    - A00 ~ E99 --> doenças

    - F00       --> Transtornos Mentais

    - G00 ~ N99 --> doenças

    - O00 ~ Q99 --> problemas na gestão/Parto e malformações

    - R00       --> doenças

    - S00 ~ T98 --> Consequências de causas externas (lesões/ traumatismos)

    - U00 ~ Y98 --> Causas externas de morbidade e de mortalidade (acidente/agressao/intoxicação/falta de comida/falta de agua/queda/tiro)

    - Z00 ~     --> doenças

In [ ]:
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[O-Q].*', 'problemas na gestão/Parto e malformações')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('R.*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[S-T].*', 'Consequências de causas externas (lesões/ traumatismos)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[U-Y].*', 'Causas externas de morbidade e de mortalidade (acidente/agressao)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('Z.*', 'Doença')